<a href="https://colab.research.google.com/github/kazimianec/TARIC/blob/master/TRCtransformersTF_BERT_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
url_TARIC_C2_descrptions = "https://raw.githubusercontent.com/kazimianec/TARIC/master/DATA/TARIC_C2_descriptions.csv"
url_TARIC_C4_descrptions ="https://raw.githubusercontent.com/kazimianec/TARIC/master/DATA/TARIC_C4_descriptions.csv"
url_TARIC_C6_descrptions ="https://raw.githubusercontent.com/kazimianec/TARIC/master/DATA/TARIC_C6_descriptions.csv"
url_TARIC_C246_descrptions = "https://raw.githubusercontent.com/kazimianec/TARIC/master/DATA/TARIC_C246_descriptions_CONCAT.csv"


In [4]:
import pandas as pd 
import io 
  
df_C2_descriptions = pd.read_csv(url_TARIC_C2_descrptions) 
df_C4_descriptions = pd.read_csv(url_TARIC_C4_descrptions)
df_C6_descriptions = pd.read_csv(url_TARIC_C6_descrptions)
df_C246_descriptions = pd.read_csv(url_TARIC_C246_descrptions)

In [5]:
for col in df_C246_descriptions.columns: 
    print(col)

CODE_2
CODE_4
CODE_6
goods_CODE_6
Description_46
Description_246
Description_46_concat
Description_246_concat


In [6]:
#sentences = df['CLASS_Description'].tolist()

sentences_C2 = df_C2_descriptions['Description_2'].tolist()
sentences_C4 = df_C4_descriptions['Description_4'].tolist()
sentences_C6 = df_C6_descriptions['Description_6'].tolist()

sentences_C246 = df_C246_descriptions['Description_246'].tolist()
sentences_C46 = df_C246_descriptions['Description_46'].tolist()

sentences_C246_concat = df_C246_descriptions['Description_246_concat'].tolist()
sentences_C46_concat = df_C246_descriptions['Description_46_concat'].tolist()

In [7]:
sentences = sentences_C2
sentences = [element.lower() for element in sentences]


In [8]:
len(sentences)

98

**Test 0. "Good descriptions"**

**Test 1. English, all words are NOT in TARIC**

**Test 2. Englsih, words are in "EXCLUDING ...", "OTHER THAN...." parts of TARIC descriptions**

**Test 3. Not obvious descriptions**

In [9]:
description = ["Sterling Silver Turquoise Art Deco Earrings"]
description = ["I Love Makeup - Nail Polish"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]

**ENCODER BERT**

In [10]:
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np

In [11]:
!pip install tensorflow_text
import tensorflow_text

In [12]:
#english_sentences = tf.constant(["dog", "Puppies are nice.", "I enjoy taking long walks along the beach with my dog."])
TARIC_sentences = tf.constant(sentences)

In [13]:
#preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
preprocessor = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")


In [14]:
#encoder = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3")
encoder = hub.load("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1")

#encoder = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/en-base/1")



In [13]:
embeddings_TARIC_long = []

In [15]:
prep = preprocessor(TARIC_sentences)
print(prep.keys())

dict_keys(['input_word_ids', 'input_mask', 'input_type_ids'])


In [16]:
prep["input_word_ids"]

<tf.Tensor: shape=(98, 128), dtype=int32, numpy=
array([[ 101, 2444, 4176, ...,    0,    0,    0],
       [ 101, 6240, 1998, ...,    0,    0,    0],
       [ 101, 3869, 1998, ...,    0,    0,    0],
       ...,
       [ 101, 2948, 1010, ...,    0,    0,    0],
       [ 101, 3719, 1010, ...,    0,    0,    0],
       [ 101, 9380, 1010, ...,    0,    0,    0]], dtype=int32)>

In [17]:
embeddings_TARIC_long_dict = encoder(prep)
# returns a dictionary with keys dict_keys(['encoder_outputs', 'sequence_output', 'default', 'pooled_output'])
#print(embeddings_TARIC_long.keys())

In [18]:
print(embeddings_TARIC_long_dict["encoder_outputs"][0].shape)
print(embeddings_TARIC_long_dict["sequence_output"][0].shape)
print(embeddings_TARIC_long_dict["sequence_output"].shape)

print(embeddings_TARIC_long_dict["pooled_output"].shape)

(98, 128, 256)
(128, 256)
(98, 128, 256)
(98, 256)


In [19]:
print(embeddings_TARIC_long_dict.keys())

dict_keys(['encoder_outputs', 'sequence_output', 'default', 'pooled_output'])


In [21]:
embeddings_TARIC_default = embeddings_TARIC_long_dict["default"]
embeddings_TARIC_encoder_outputs = embeddings_TARIC_long_dict["encoder_outputs"]
embeddings_TARIC_sequence_output = embeddings_TARIC_long_dict["sequence_output"]
embeddings_TARIC_pooled_output = embeddings_TARIC_long_dict["pooled_output"]

embeddings_TARIC_long = embeddings_TARIC_default

In [26]:
# cycle is less efficient comparing to vectorised processing, but uses less memory
# and vector version would be just:
#embeddings_TARIC_long = encoder(prep)["default"]
#embeddings_TARIC_long = encoder(preprocessor(TARIC_sentences))
#for sentence_id in sentences:
# take a care on square brackets. It expects a tensor, not just a string. So, reshaping sentence_id
#  embeddings_TARIC_id = encoder(preprocessor(tf.constant([sentence_id])))["default"]
#  embeddings_TARIC_long.append(embeddings_TARIC_id)

In [18]:
#embeddings_TARIC_long = tf.squeeze(embeddings_TARIC_long)
print(embeddings_TARIC_long.shape)

(98, 768)


In [ ]:
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-lite/2")
#embed = hub.load ("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3")


In [ ]:
# = embed(sentences)

In [33]:


description = ["Sterling Silver Turquoise Art Deco Earrings"]




description = ["Vyriškos kelnės"]

description = ["I Love Makeup - Nail Polish"] #yes!
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"] #no
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["Fosmon Apple iPhone 4 / iPhone 4G USB Sync Charge Data Cable with USB Auto Car Charger and USB Home Travel Charger"] #SUPER matches S3
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"] # not


description = ["HTC HD7 Crowned Heart Phone Protector Cover Case"] ## bullshit is both cases - bert and S3
description = ["Fosmon Apple iPhone 4 / iPhone 4G USB Sync Charge Data Cable with USB Auto Car Charger and USB Home Travel Charger"] #SUPER matches S3 

description = ["Sterling Silver Turquoise Art Deco Earrings"]


description = ["Fosmon Apple iPhone 4 / iPhone 4G USB Sync Charge Data Cable with USB Auto Car Charger and USB Home Travel Charger"] #SUPER matches S3 



description = ["Jolly Jumper Pashmama Nursing Cover - Tan "]
description = ["HTC HD7 Crowned Heart Phone Protector Cover Case"]





description = ["Jolly Jumper Pashmama Nursing Cover - Tan"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]

description = ["Fosmon Apple iPhone 4 / iPhone 4G USB Sync Charge Data Cable with USB Auto Car Charger and USB Home Travel Charger"]

description = ["Meat beef"]
description = ["I Love Makeup - Nail Polish"] #yes!

TARIC_description = tf.constant(description)
embeddings_description = encoder(preprocessor(TARIC_description))["default"]


cosine=np.dot(embeddings_TARIC_long, np.transpose(embeddings_description))
cosine=np.inner(embeddings_TARIC_long, embeddings_description)
#print(cosine)
ind_max = tf.constant(tf.argmax(cosine)).numpy()[0]
top_ind = tf.math.top_k(np.transpose(cosine), k=5, sorted=True, name=None).indices.numpy()[0]
top_value = tf.math.top_k(np.transpose(cosine), k=5, sorted=True, name=None).values.numpy()[0]
print(cosine[ind_max])
#print(classes_6[ind_max])
print(top_value[0], sentences[top_ind[0]])
print(top_value[1], sentences[top_ind[1]])
print(top_value[2], sentences[top_ind[2]])
print(top_value[3], sentences[top_ind[3]])
print(top_value[4], sentences[top_ind[4]])

[103.2975]
103.2975 starches; inulin
102.33307 oats
101.5146 rye
99.72162 carboxylic acids with additional oxygen function and their anhydrides, halides, peroxides and peroxyacids; their halogenated, sulphonated, nitrated or nitrosated derivatives
96.90134 other manufactured tobacco and manufactured tobacco substitutes; 'homogenised' or 'reconstituted' tobacco; tobacco extracts and essences


In [ ]:
cosine.shape

(98, 1)